# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [1]:
import fish_data as fd
import numpy as np
import tensorflow as tf

#### Helper functions

In [2]:
help(fd)

Help on module fish_data:

NAME
    fish_data

DESCRIPTION
    fish_data module contains the helper functions for the model build of the
    Nature Conservancy Fisheries Kaggle Competition.
    
    Dependencies:
        * numpy as np
        * scipy.ndimage as ndimage
        * scipy.misc as misc

FUNCTIONS
    generate_epoch_set_list_and_label_array(min_each)
        Function to generate a list of filenames to be used for each training
        epoch.  Some files will be used multiple times in order that each fish is
        drawn into a training batch an equivalent number of times.
    
    mutate_image(image)
        Receives an image array and returns an image array with a random set of
            distortions.
        Distortions include:
            * flip horizontally (0.5)
            * flip vertically (0.5)
            * slight distortion of coloring (0.5 for any distortion at all, plus
                random float for amount of distortion)
            * random horizontal shif